In [5]:
import pandas as pd
from sodapy import Socrata

In [10]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("fhrw-4uyv", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [8]:
results_df.head()

,address_type,agency,agency_name,bbl,borough,city,closed_date,community_board,complaint_type,created_date,...,park_facility_name,resolution_action_updated_date,resolution_description,status,street_name,taxi_company_borough,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
0,ADDRESS,NYPD,New York City Police Department,1009190004,MANHATTAN,NEW YORK,2015-11-21T15:16:52.000,06 MANHATTAN,Noise - Street/Sidewalk,2015-11-21T09:20:34.000,...,Unspecified,2015-11-21T15:16:52.000,The Police Department responded to the complai...,Closed,3 AVENUE,NaN,NaN,32037169,990867,211966
1,ADDRESS,NYPD,New York City Police Department,1018640009,MANHATTAN,NEW YORK,2015-11-22T02:34:50.000,07 MANHATTAN,Noise - Residential,2015-11-22T02:04:02.000,...,Unspecified,2015-11-22T02:34:50.000,The Police Department responded to the complai...,Closed,WEST 109 STREET,NaN,NaN,32037170,994669,231311
2,ADDRESS,NYPD,New York City Police Department,1019880118,MANHATTAN,NEW YORK,2015-11-21T13:19:53.000,09 MANHATTAN,Noise - Residential,2015-11-21T10:49:20.000,...,Unspecified,2015-11-21T13:19:53.000,The Police Department responded to the complai...,Closed,WEST 136 STREET,NaN,NaN,32037171,997415,237975
3,ADDRESS,NYPD,New York City Police Department,3045960049,BROOKLYN,BROOKLYN,2015-11-21T08:26:22.000,17 BROOKLYN,Noise - Residential,2015-11-21T07:48:54.000,...,Unspecified,2015-11-21T08:26:08.000,The Police Department responded to the complai...,Closed,EAST 93 STREET,NaN,NaN,32037172,1004522,180633
4,INTERSECTION,DSNY,BCC - Staten Island,NaN,STATEN ISLAND,STATEN ISLAND,2015-11-21T12:00:00.000,01 STATEN ISLAND,Derelict Vehicles,2015-11-21T10:59:00.000,...,Unspecified,2015-11-21T10:59:00.000,The Department of Sanitation removed the items.,Pending,NaN,NaN,NaN,32037173,951469,169743


In [12]:
data = results_df

In [14]:
data.dtypes

address_type                      object
agency                            object
agency_name                       object
bbl                               object
borough                           object
city                              object
closed_date                       object
community_board                   object
complaint_type                    object
created_date                      object
cross_street_1                    object
cross_street_2                    object
descriptor                        object
due_date                          object
facility_type                     object
incident_address                  object
incident_zip                      object
intersection_street_1             object
intersection_street_2             object
latitude                          object
location                          object
location_type                     object
longitude                         object
open_data_channel_type            object
park_borough    

In [15]:
data.agency.value_counts()

HPD      994
NYPD     567
DEP      133
DSNY      93
DOT       84
DOB       40
DOHMH     30
DPR       25
TLC       19
DHS        8
DCA        6
EDC        1
Name: agency, dtype: int64

In [18]:
data.complaint_type.value_counts()

HEAT/HOT WATER                             475
Noise - Residential                        267
UNSANITARY CONDITION                       137
PLUMBING                                    98
Noise                                       98
Blocked Driveway                            83
PAINT/PLASTER                               72
Noise - Commercial                          69
Illegal Parking                             61
DOOR/WINDOW                                 51
Missed Collection (All Materials)           51
Noise - Street/Sidewalk                     45
FLOORING/STAIRS                             40
WATER LEAK                                  37
ELECTRIC                                    31
Street Condition                            30
Broken Muni Meter                           28
GENERAL                                     24
Sanitation Condition                        16
Water System                                15
Taxi Complaint                              14
General Const